In [1]:
from sklearn.model_selection import train_test_split
import tensorflow as tf
from sklearn import datasets

In [2]:
# import some data to play with
iris = datasets.load_iris()
X = iris.data  # we only take the first two features.
y = iris.target

In [3]:
print(f'Sepal shape {X[:, 0].shape}')
print(f'irises shape {y.shape}')

Sepal shape (150,)
irises shape (150,)


In [4]:
label = ['setosa', 'versicolor', 'virginica']

In [5]:
# chia train và test
x_train, x_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [6]:
print('shape X train numbers : '+ str(x_train.shape))
print('shape y train result : '+ str(y_train.shape))
print('-----------------------------------------')
print('shape X test numbers : '+ str(x_test.shape))
print('shape y test result : '+ str(y_test.shape))

shape X train numbers : (120, 4)
shape y train result : (120,)
-----------------------------------------
shape X test numbers : (30, 4)
shape y test result : (30,)


In [7]:
# chuẩn hóa dữ liệu
mean = x_train.mean(axis=0)
std = x_train.std(axis=0)

x_train = (x_train - mean) / std

In [8]:
# xây dựng mô hình
def create_model():
    model = tf.keras.Sequential([
        tf.keras.layers.Dense(64, activation='relu', input_shape=(4,)),
        tf.keras.layers.Dropout(0.2),
        tf.keras.layers.Dense(128, activation='relu', input_shape=(2,)),
        tf.keras.layers.Dropout(0.2),
        tf.keras.layers.Dense(256, activation='relu', input_shape=(2,)),
        tf.keras.layers.Dense(3, activation='softmax')
    ])

    # Compile mô hình
    model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
    
    return model

In [9]:
model = create_model()
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 64)                320       
                                                                 
 dropout (Dropout)           (None, 64)                0         
                                                                 
 dense_1 (Dense)             (None, 128)               8320      
                                                                 
 dropout_1 (Dropout)         (None, 128)               0         
                                                                 
 dense_2 (Dense)             (None, 256)               33024     
                                                                 
 dense_3 (Dense)             (None, 3)                 771       
                                                                 
Total params: 42,435
Trainable params: 42,435
Non-traina

In [10]:
y_train_categorical = tf.keras.utils.to_categorical(y_train)

In [11]:
# Thiết lập hàm call back Early Stopping
callback = tf.keras.callbacks.EarlyStopping(monitor='val_loss', min_delta=0.01, mode="auto", patience=10, restore_best_weights=True)

# Huấn luyện mô hình
history = model.fit(x_train, y_train_categorical, epochs=150, batch_size=128, validation_split=0.2, shuffle=True, callbacks = callback)

Epoch 1/150
1/1 [==============================] - 1s 983ms/step - loss: 1.1552 - accuracy: 0.3229 - val_loss: 1.0492 - val_accuracy: 0.6250
Epoch 2/150
1/1 [==============================] - 0s 43ms/step - loss: 1.0810 - accuracy: 0.4583 - val_loss: 0.9894 - val_accuracy: 0.8333
Epoch 3/150
1/1 [==============================] - 0s 37ms/step - loss: 1.0136 - accuracy: 0.7083 - val_loss: 0.9323 - val_accuracy: 0.8333
Epoch 4/150
1/1 [==============================] - 0s 41ms/step - loss: 0.9541 - accuracy: 0.8021 - val_loss: 0.8787 - val_accuracy: 0.8750
Epoch 5/150
1/1 [==============================] - 0s 41ms/step - loss: 0.9075 - accuracy: 0.8021 - val_loss: 0.8273 - val_accuracy: 0.8750
Epoch 6/150
1/1 [==============================] - 0s 34ms/step - loss: 0.8503 - accuracy: 0.7917 - val_loss: 0.7782 - val_accuracy: 0.8750
Epoch 7/150
1/1 [==============================] - 0s 42ms/step - loss: 0.7927 - accuracy: 0.8021 - val_loss: 0.7316 - val_accuracy: 0.8750
Epoch 8/150
1/1 [==

1/1 [==============================] - 0s 32ms/step - loss: 0.1561 - accuracy: 0.9479 - val_loss: 0.2113 - val_accuracy: 0.9583
Epoch 60/150
1/1 [==============================] - 0s 32ms/step - loss: 0.1441 - accuracy: 0.9375 - val_loss: 0.2123 - val_accuracy: 0.9583
Epoch 61/150
1/1 [==============================] - 0s 34ms/step - loss: 0.1727 - accuracy: 0.9271 - val_loss: 0.2153 - val_accuracy: 0.9583
Epoch 62/150
1/1 [==============================] - 0s 32ms/step - loss: 0.1541 - accuracy: 0.9479 - val_loss: 0.2249 - val_accuracy: 0.9583
Epoch 63/150
1/1 [==============================] - 0s 35ms/step - loss: 0.1016 - accuracy: 0.9792 - val_loss: 0.2387 - val_accuracy: 0.9583
Epoch 64/150
1/1 [==============================] - 0s 31ms/step - loss: 0.1130 - accuracy: 0.9583 - val_loss: 0.2601 - val_accuracy: 0.9167
Epoch 65/150
1/1 [==============================] - 0s 33ms/step - loss: 0.1167 - accuracy: 0.9583 - val_loss: 0.2797 - val_accuracy: 0.9167
Epoch 66/150
1/1 [========

In [ ]:
import matplotlib.pyplot as plt
plt.plot(history.history['loss'], label='loss')
plt.plot(history.history['val_loss'], label = 'val_loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.legend(loc='lower right')
plt.show()

In [12]:
import numpy as np
# hàm dự đoán
def predict(model, x):
    x = (x - mean) / std
    y_pred = model.predict(x) # dự đoán xác suất cho mỗi loài hoa
    return np.argmax(y_pred, axis=1) # chọn loài hoa có xác suất cao nhất

In [14]:
# dự đoán tập test
y_pred = predict(model, x_test)
for i, pred in enumerate(y_pred):
    print(f'pred: {label[pred]}, y = {label[y_test[i]]}')

1/1 [==============================] - 0s 20ms/step
pred: versicolor, y = versicolor
pred: setosa, y = setosa
pred: virginica, y = virginica
pred: virginica, y = versicolor
pred: versicolor, y = versicolor
pred: setosa, y = setosa
pred: versicolor, y = versicolor
pred: virginica, y = virginica
pred: virginica, y = versicolor
pred: versicolor, y = versicolor
pred: virginica, y = virginica
pred: setosa, y = setosa
pred: setosa, y = setosa
pred: setosa, y = setosa
pred: setosa, y = setosa
pred: versicolor, y = versicolor
pred: virginica, y = virginica
pred: versicolor, y = versicolor
pred: versicolor, y = versicolor
pred: virginica, y = virginica
pred: setosa, y = setosa
pred: virginica, y = virginica
pred: setosa, y = setosa
pred: virginica, y = virginica
pred: virginica, y = virginica
pred: virginica, y = virginica
pred: virginica, y = virginica
pred: virginica, y = virginica
pred: setosa, y = setosa
pred: setosa, y = setosa
